<a href="https://colab.research.google.com/github/claudio1975/Medium-blog/blob/master/Summarization_tutorial/Summarization_insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Prepare workspace

In [1]:
! pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.6 MB/s eta 0:00:00


In [2]:
! pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.3/426.3 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 2.2 MB/s eta 0:00:00


In [3]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 20.4 MB/s eta 0:00:00


In [4]:
from langchain import OpenAI, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import T5Tokenizer, T5ForConditionalGeneration

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
import os
os.environ["OPENAI_API_KEY"] = ""

### Load document

In [6]:
# without titles and subtitles 
sample_doc = """
You must report to Us immediately any
accident, injury, loss or damage which
may give rise to a claim under this Policy.
All incidents must be reported to our
Emergency Services phone number:
Republic of Ireland 1890 48 48 48
Northern Ireland or United Kingdom
00353 1 6133666.

In the event of an accident You should
obtain the following information:
1. The names, contact details and
vehicle details of all parties
involved.
2. The insurance details including the
Policy number of all parties
involved.
3. Details of any witnesses to the
incident or members of An Garda
Siochana / Police that attended the
scene of the accident.

1. Not admit responsibility, sign any
statement or negotiate the
settlement of any claim, without the
written agreement of Allianz.
2. Complete any form(s) We may send
You.
3. Give Us all information and
assistance required.
4. Notify Us immediately of any
impending prosecution, inquest or
fatal inquiry, writ or summons.
5. Send Us, as soon as possible, any
writ or summons, letter or other
documents You may receive.
6. The registration and insurance
details of Your Car should be
provided to any other party involved
and also An Garda/Police, if
requested.
7. If any person is injured, the accident
must be reported to An
Garda/Police, whether they attend
the scene of not.
If You do not do so, We reserve the
right not to pay a claim. We are
entitled, at any stage during any claim,
to take over and conduct the defence
or settlement of the claim, and, at our
discretion, to pursue the claim for our
own benefit in the name of any person
insured.
"""


### Extractive summarization with NLTK

In [7]:
# code from https://www.geeksforgeeks.org/python-text-summarizer/
# Tokenizing the text
words = word_tokenize(sample_doc)
   
# Creating a frequency table to keep the 
# score of each word
   
freqTable = dict()
for word in words:
    word = word.lower()
    if word in stop_words:
        continue
    if word in freqTable:
        freqTable[word] += 1
    else:
        freqTable[word] = 1
   
# Creating a dictionary to keep the score
# of each sentence
sentences = sent_tokenize(sample_doc)
sentenceValue = dict()
   
for sentence in sentences:
    for word, freq in freqTable.items():
        if word in sentence.lower():
            if sentence in sentenceValue:
                sentenceValue[sentence] += freq
            else:
                sentenceValue[sentence] = freq
   
   
   
sumValues = 0
for sentence in sentenceValue:
    sumValues += sentenceValue[sentence]
   
# Average value of a sentence from the original text
   
average = int(sumValues / len(sentenceValue))
   
# Storing sentences into our summary.
NLTK_summary = ''
for sentence in sentences:
    if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)):
        NLTK_summary += " " + sentence


In [20]:
print(NLTK_summary)

 
You must report to Us immediately any
accident, injury, loss or damage which
may give rise to a claim under this Policy. All incidents must be reported to our
Emergency Services phone number:
Republic of Ireland 1890 48 48 48
Northern Ireland or United Kingdom
00353 1 6133666. The names, contact details and
vehicle details of all parties
involved. Not admit responsibility, sign any
statement or negotiate the
settlement of any claim, without the
written agreement of Allianz. Notify Us immediately of any
impending prosecution, inquest or
fatal inquiry, writ or summons. Send Us, as soon as possible, any
writ or summons, letter or other
documents You may receive. The registration and insurance
details of Your Car should be
provided to any other party involved
and also An Garda/Police, if
requested. If any person is injured, the accident
must be reported to An
Garda/Police, whether they attend
the scene of not. We are
entitled, at any stage during any claim,
to take over and conduct the d

### Abstractive summarization with T5 model from Hugging Face

In [9]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base', return_dict=True)


/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [10]:
inputs = tokenizer.encode("summarize: " + sample_doc,
                          return_tensors='pt',
                          truncation=True)

In [11]:
summary_ids = model.generate(inputs, max_length=500, min_length=100,num_beams=4, early_stopping=True)

In [12]:
T5_summary = tokenizer.decode(summary_ids[0],skip_special_tokens=True)
T5_summary

'all incidents must be reported to our Emergency Services phone number. details of all parties involved, insurance details including Policy number. not admit responsibility, sign any statement or negotiate settlement of claim. notify us of impending prosecution, inquest or fatal inquiry, writ or summons, letter or other documents. if any person is injured, the accident must be reported to An Garda/Police, whether they attend the scene of not, if requested.'

### Abstractive summarization with GPT-3 from OpenAI using LangChain

In [13]:
llm_model = OpenAI(temperature=0, max_tokens=500)
text_splitter = CharacterTextSplitter()

In [14]:
claims_texts = text_splitter.split_text(sample_doc)

In [15]:
claiams_docs = [Document(page_content=t) for t in claims_texts]

In [16]:
chain = load_summarize_chain(llm_model)
LangChain_summary = chain.run(claiams_docs)

In [18]:
LangChain_summary

" In the event of an accident, you must report it to Allianz's Emergency Services phone number and obtain the names, contact details, vehicle details, and insurance details of all parties involved, as well as any witnesses or police that attended the scene. You must not admit responsibility, sign any statement, or negotiate the settlement of any claim without Allianz's written agreement. You must also provide the registration and insurance details of your car to any other party involved and the police, if requested. If any person is injured, the accident must be reported to the police, even if they do not attend the scene. If you do not do so, Allianz reserves the right not to pay a claim."